## Medicare Part D Data by Provider

Aggregate by Prscrbr_NPI

Keep prescriber NPI and info about name, location

aggregate other data into dictionaries


In [8]:
import requests
import pandas as pd
from collections import defaultdict
import time

### Test on 20K rows

In [11]:
def get_paginated_data(base_url, size=5000, max_records=1000000):
    offset = 0
    all_data = []
    
    while offset < max_records:
        res = requests.get(f"{base_url}?size={size}&offset={offset}")
        data = res.json()
        
        if not data:
            break
        
        all_data.extend(data)
        offset += size

    return pd.DataFrame(all_data)

def aggregate_by_npi(data):
    # Dictionary to aggregate data per NPI
    aggregated_data = defaultdict(lambda: defaultdict(dict))
    
    for _, row in data.iterrows():
        npi = row['Prscrbr_NPI']
        
        # Add basic prescriber info (this remains unique per NPI)
        aggregated_data[npi]['Prscrbr_Last_Org_Name'] = row['Prscrbr_Last_Org_Name']
        aggregated_data[npi]['Prscrbr_First_Name'] = row['Prscrbr_First_Name']
        aggregated_data[npi]['Prscrbr_City'] = row['Prscrbr_City']
        aggregated_data[npi]['Prscrbr_State_Abrvtn'] = row['Prscrbr_State_Abrvtn']
        aggregated_data[npi]['Prscrbr_State_FIPS'] = row['Prscrbr_State_FIPS']
        aggregated_data[npi]['Prscrbr_Type'] = row['Prscrbr_Type']
        aggregated_data[npi]['Prscrbr_Type_Src'] = row['Prscrbr_Type_Src']
        
        # Aggregate prescription-related info as dictionaries with indexed keys
        index = len(aggregated_data[npi]['Brnd_Name']) + 1  # Start at 1, increment for each new entry
        
        aggregated_data[npi]['Brnd_Name'][index] = row['Brnd_Name']
        aggregated_data[npi]['Gnrc_Name'][index] = row['Gnrc_Name']
        aggregated_data[npi]['Tot_Clms'][index] = row['Tot_Clms']
        aggregated_data[npi]['Tot_30day_Fills'][index] = row['Tot_30day_Fills']
        aggregated_data[npi]['Tot_Day_Suply'][index] = row['Tot_Day_Suply']
        aggregated_data[npi]['Tot_Drug_Cst'][index] = row['Tot_Drug_Cst']
        aggregated_data[npi]['Tot_Benes'][index] = row['Tot_Benes']
        aggregated_data[npi]['GE65_Sprsn_Flag'][index] = row['GE65_Sprsn_Flag']
        aggregated_data[npi]['GE65_Tot_Clms'][index] = row['GE65_Tot_Clms']
        aggregated_data[npi]['GE65_Tot_30day_Fills'][index] = row['GE65_Tot_30day_Fills']
        aggregated_data[npi]['GE65_Tot_Drug_Cst'][index] = row['GE65_Tot_Drug_Cst']
        aggregated_data[npi]['GE65_Tot_Day_Suply'][index] = row['GE65_Tot_Day_Suply']
        aggregated_data[npi]['GE65_Bene_Sprsn_Flag'][index] = row['GE65_Bene_Sprsn_Flag']
        aggregated_data[npi]['GE65_Tot_Benes'][index] = row['GE65_Tot_Benes']
    
    # Convert the aggregated data to a DataFrame
    aggregated_df = pd.DataFrame.from_dict(aggregated_data, orient='index')
    
    return aggregated_df

In [12]:
# Base URL for the dataset (Medicare Part D Providers)
base_url = "https://data.cms.gov/data-api/v1/dataset/9552739e-3d05-4c1b-8eff-ecabf391e2e5/data"

# Measure the time taken
start_time = time.time()

# Step 1: Fetch the first 20,000 rows
data = get_paginated_data(base_url, size=5000, max_records=20000)

# Step 2: Aggregate data by NPI
aggregated_data = aggregate_by_npi(data)

# Step 3: Measure the end time
end_time = time.time()

# Step 4: Display the aggregated columns (prescription-related info)
aggregated_columns = [
    'Brnd_Name', 'Gnrc_Name', 'Tot_Clms', 'Tot_30day_Fills',
    'Tot_Day_Suply', 'Tot_Drug_Cst', 'Tot_Benes', 'GE65_Sprsn_Flag',
    'GE65_Tot_Clms', 'GE65_Tot_30day_Fills', 'GE65_Tot_Drug_Cst',
    'GE65_Tot_Day_Suply', 'GE65_Bene_Sprsn_Flag', 'GE65_Tot_Benes'
]

# Show the head of aggregated columns
print(aggregated_data[aggregated_columns].head())

# Step 5: Print the total time taken
print(f"Time taken to complete: {end_time - start_time} seconds")

                                                    Brnd_Name  \
1003000126  {1: 'Amlodipine Besylate', 2: 'Atorvastatin Ca...   
1003000142  {1: 'Acetaminophen-Codeine', 2: 'Amitriptyline...   
1003000167             {1: 'Amoxicillin', 2: 'Dexamethasone'}   
1003000423  {1: 'Clobetasol Propionate', 2: 'Estradiol', 3...   
1003000480                                  {1: 'Omeprazole'}   

                                                    Gnrc_Name  \
1003000126  {1: 'Amlodipine Besylate', 2: 'Atorvastatin Ca...   
1003000142  {1: 'Acetaminophen With Codeine', 2: 'Amitript...   
1003000167             {1: 'Amoxicillin', 2: 'Dexamethasone'}   
1003000423  {1: 'Clobetasol Propionate', 2: 'Estradiol', 3...   
1003000480                                  {1: 'Omeprazole'}   

                                                     Tot_Clms  \
1003000126  {1: '19', 2: '11', 3: '15', 4: '16', 5: '12', ...   
1003000142  {1: '17', 2: '13', 3: '33', 4: '27', 5: '45', ...   
1003000167             

### Get all Data



In [13]:
def get_paginated_data(base_url, size=100000):
    offset = 0
    all_data = []
    
    while True:
        res = requests.get(f"{base_url}?size={size}&offset={offset}")
        data = res.json()
        
        if not data:
            break
        
        all_data.extend(data)
        offset += size
        print(f"Fetched {offset} records...")
    
    return pd.DataFrame(all_data)

def aggregate_by_npi(data):
    # Dictionary to aggregate data per NPI
    aggregated_data = defaultdict(lambda: defaultdict(dict))
    
    for _, row in data.iterrows():
        npi = row['Prscrbr_NPI']
        
        # Add basic prescriber info (this remains unique per NPI)
        aggregated_data[npi]['Prscrbr_Last_Org_Name'] = row['Prscrbr_Last_Org_Name']
        aggregated_data[npi]['Prscrbr_First_Name'] = row['Prscrbr_First_Name']
        aggregated_data[npi]['Prscrbr_City'] = row['Prscrbr_City']
        aggregated_data[npi]['Prscrbr_State_Abrvtn'] = row['Prscrbr_State_Abrvtn']
        aggregated_data[npi]['Prscrbr_State_FIPS'] = row['Prscrbr_State_FIPS']
        aggregated_data[npi]['Prscrbr_Type'] = row['Prscrbr_Type']
        aggregated_data[npi]['Prscrbr_Type_Src'] = row['Prscrbr_Type_Src']
        
        # Aggregate prescription-related info as dictionaries with indexed keys
        index = len(aggregated_data[npi]['Brnd_Name']) + 1  # Start at 1, increment for each new entry
        
        aggregated_data[npi]['Brnd_Name'][index] = row['Brnd_Name']
        aggregated_data[npi]['Gnrc_Name'][index] = row['Gnrc_Name']
        aggregated_data[npi]['Tot_Clms'][index] = row['Tot_Clms']
        aggregated_data[npi]['Tot_30day_Fills'][index] = row['Tot_30day_Fills']
        aggregated_data[npi]['Tot_Day_Suply'][index] = row['Tot_Day_Suply']
        aggregated_data[npi]['Tot_Drug_Cst'][index] = row['Tot_Drug_Cst']
        aggregated_data[npi]['Tot_Benes'][index] = row['Tot_Benes']
        aggregated_data[npi]['GE65_Sprsn_Flag'][index] = row['GE65_Sprsn_Flag']
        aggregated_data[npi]['GE65_Tot_Clms'][index] = row['GE65_Tot_Clms']
        aggregated_data[npi]['GE65_Tot_30day_Fills'][index] = row['GE65_Tot_30day_Fills']
        aggregated_data[npi]['GE65_Tot_Drug_Cst'][index] = row['GE65_Tot_Drug_Cst']
        aggregated_data[npi]['GE65_Tot_Day_Suply'][index] = row['GE65_Tot_Day_Suply']
        aggregated_data[npi]['GE65_Bene_Sprsn_Flag'][index] = row['GE65_Bene_Sprsn_Flag']
        aggregated_data[npi]['GE65_Tot_Benes'][index] = row['GE65_Tot_Benes']
    
    # Convert the aggregated data to a DataFrame
    aggregated_df = pd.DataFrame.from_dict(aggregated_data, orient='index')
    
    return aggregated_df

In [14]:
# Base URL for the dataset (Medicare Part D Providers)
base_url = "https://data.cms.gov/data-api/v1/dataset/9552739e-3d05-4c1b-8eff-ecabf391e2e5/data"

# Measure the time taken
start_time = time.time()

# Step 1: Fetch all available rows
data = get_paginated_data(base_url)

# Step 2: Aggregate data by NPI
aggregated_data = aggregate_by_npi(data)

# Step 3: Measure the end time
end_time = time.time()

# Step 4: Display the aggregated columns (prescription-related info)
aggregated_columns = [
    'Brnd_Name', 'Gnrc_Name', 'Tot_Clms', 'Tot_30day_Fills',
    'Tot_Day_Suply', 'Tot_Drug_Cst', 'Tot_Benes', 'GE65_Sprsn_Flag',
    'GE65_Tot_Clms', 'GE65_Tot_30day_Fills', 'GE65_Tot_Drug_Cst',
    'GE65_Tot_Day_Suply', 'GE65_Bene_Sprsn_Flag', 'GE65_Tot_Benes'
]

# Show the head of aggregated columns
print(aggregated_data[aggregated_columns].head())

# Step 5: Print the total time taken
print(f"Time taken to complete: {end_time - start_time} seconds")

Fetched 5000 records...
Fetched 10000 records...
Fetched 15000 records...
Fetched 20000 records...
Fetched 25000 records...
Fetched 30000 records...
Fetched 35000 records...
Fetched 40000 records...
Fetched 45000 records...
Fetched 50000 records...
Fetched 55000 records...
Fetched 60000 records...
Fetched 65000 records...
Fetched 70000 records...
Fetched 75000 records...
Fetched 80000 records...
Fetched 85000 records...
Fetched 90000 records...
Fetched 95000 records...
Fetched 100000 records...
Fetched 105000 records...
Fetched 110000 records...
Fetched 115000 records...
Fetched 120000 records...
Fetched 125000 records...
Fetched 130000 records...
Fetched 135000 records...
Fetched 140000 records...
Fetched 145000 records...
Fetched 150000 records...
Fetched 155000 records...
Fetched 160000 records...
Fetched 165000 records...
Fetched 170000 records...
Fetched 175000 records...
Fetched 180000 records...
Fetched 185000 records...
Fetched 190000 records...
Fetched 195000 records...
Fetche

KeyboardInterrupt: 

In [17]:
print(len(aggregated_data))

918
